# Plots for section 5

In [ ]:
#TODO: Try to replicate some plots from Ruhnau Paper (have another look at it)

### Imports

In [ ]:
import pandas as pd
import matplotlib
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import geopandas as gpd

from utils import market_values, market_values_links, market_values_by_time_index, market_values_links_con, congestion_rent_link, convert_ISO_3166_2_to_1, generation, generation_links, market_values_storage_units, generation_storage_units

# imported own functions
from utils import market_values, market_values_by_time_index, nodal_balance, capacity, capacity_links, capacity_storage_units, get_condense_sum

# imported own definitions
from utils import carrier_colors, carrier_renaming, resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal

# general variables
font1 = {'fontname':'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

In [ ]:
# Network imports
n_no =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
n_h2 =pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

In [ ]:
# Notebook Definitions

c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger", "solar thermal"]

In [ ]:
# Notebook Functions

def get_df(df_no, df_h2, carriers):
    result = pd.DataFrame(index = ["no H2 network", "H2 network"])

    for c in carriers:
        result.loc["no H2 network" , c] = df_no[c].values
        result.loc["H2 network" , c] = df_h2[c].values
    return result

In [ ]:
# Regions

onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

### General
- round trip efficiency of hydrogen and batter storage (grafik mit power point machen) strom -> electrolyse -> storage -> fuel cell -> strom


In [ ]:
n_no.links[n_no.links.carrier == "H2 Electrolysis"].efficiency.unique() # Electrolysis: 68 % efficiency
n_no.links[n_no.links.carrier == "H2 Fuel Cell"].efficiency.unique() # Fuel Cell: 50 % efficiency
n_no.links[n_no.links.carrier == "battery charger"].efficiency.unique() # Fuel Cell: 0.979796 efficiency
n_no.links[n_no.links.carrier == "battery discharger"].efficiency.unique() # Fuel Cell: 0.979796 efficiency

# storage losses?

![](../../../../Pictures/Screenshots/Screenshot_20230219_110123.png)
Colbertado

### Capacity

In [ ]:
cap_no = pd.DataFrame(index=n_no.buses.location.unique())
cap_h2 = pd.DataFrame(index=n_h2.buses.location.unique())

for n, df in zip([n_no, n_h2], [cap_no,cap_h2]):

    for c in n.generators.carrier.unique():
        # capacity in GW
        df[c] = capacity(n, c) / 1000

    for c in n.links.carrier.unique():
        # capacity in GW
        df[c] = capacity_links(n, c) / 1000

    for c in n.storage_units.carrier.unique():
        # capacity in GW
        df[c] = capacity_storage_units(n, c) / 1000

# condense groups
cap_no = get_condense_sum(cap_no, c1_groups, c1_groups_name)
cap_h2 = get_condense_sum(cap_h2, c1_groups, c1_groups_name)
# rename unhandy column names
cap_no.rename(columns=carrier_renaming, inplace=True)
cap_h2.rename(columns=carrier_renaming, inplace=True)
# take sum
cap_no_sum = pd.DataFrame(cap_no.sum().sort_values(ascending=False)).transpose()
cap_h2_sum = pd.DataFrame(cap_h2.sum().sort_values(ascending=False)).transpose()

In [ ]:
# electrolyzer capcity
carriers = ["H2 Electrolysis"]
get_df(cap_no_sum, cap_h2_sum, carriers)

In [ ]:
# H2 storage capcity
index_h2stores_no = n_no.stores[n_no.stores.carrier == "H2"].index

In [ ]:
n_no.stores[n_no.stores.carrier == "H2"]

In [ ]:
# Optimised nominal energy capacity outputed by OPF.
df = n_no.stores.e_nom_opt[index_h2stores_no]
df.index = df.index.map(n_no.stores.bus).map(n_no.buses.location)
onshore_regions["h2_cap_no"] = df

In [ ]:
plt.plot(df)

In [ ]:
fig = plt.figure(figsize=(12, 8))
crs = ccrs.EqualEarth()

ax = plt.axes(projection=ccrs.EqualEarth())
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')
ax.set_title("h2_cap", fontsize=16, **font1)

onshore_regions.to_crs(crs.proj4_init).plot(column="h2_cap_no",
                     ax=ax,
                     cmap=plt.get_cmap("magma_r"),
                     linewidth=0.05,
                     edgecolor = 'grey',
                     legend=True,
                     legend_kwds={'label':"MWh",
                        'orientation': "vertical"})

plt.show()

In [ ]:
# Why are the hydrogen capacities located there?
# calc Correlation with wind, solar and other power plant capacity (e.g.

### Generation and Consumption

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [n_no, n_h2]:

    carrier = ["H2"]
    nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
    # convert from MW to GW
    nb = nb.unstack(level=[1]) / 1000

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==n_no:
        ax_gen=axs[0, 0]
        ax_con=axs[0, 1]
        title_gen=f"Generation: {carrier} (no H2 network)"
        title_con=f"Consumption: {carrier} (no H2 network)"

    elif n==n_h2:
        ax_gen=axs[1, 0]
        ax_con=axs[1, 1]
        title_gen=f"Generation: {carrier} (with H2 network)"
        title_con=f"Consumption: {carrier} (with H2 network)"

    # generation
    c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, colors=c, startangle=0, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg, colors=c, startangle=0, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=2)
plt.show()

# fig.savefig(f"{PLOT_DIR}01_general/5.1_hydrogen_overview/gen_con_h2.png")

### Generation

In [ ]:
# how long are the electrolyzers running
# load duration curve
# capacity factors
# pie chart of which electricity (from which source?) is used for electrolysers (is that possible)

![](../../../../Pictures/Screenshots/Screenshot_20230215_163455.png)

In [ ]:
# get generation of all carriers
gen_no = pd.DataFrame(index=n_no.generators_t.p.index)
gen_h2 = pd.DataFrame(index=n_h2.generators_t.p.index)

for n, gen in zip([n_no, n_h2], [gen_no, gen_h2]):

    for carrier in n.generators.carrier.unique():
        gen[carrier]= n.generators_t.p.loc[:, n.generators.carrier == carrier].mean(axis=1)

    for carrier in n.links.carrier.unique():
        gen[carrier]= abs(n.links_t.p1.loc[:, n.links.carrier == carrier]).mean(axis=1)

    for carrier in n.storage_units.carrier.unique():
        gen[carrier]= n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier].mean(axis=1)

gen_no.head()
gen_h2.head()

In [ ]:
# why is this in a whole other magnitude than capacity?
# does sorting the whole df make sense? shouldtn the be sorted seperately?

df = pd.DataFrame(gen_no["H2 Electrolysis"].sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
df["H2 Electrolysis no h2"] = pd.DataFrame(gen_h2["H2 Electrolysis"].sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
df.columns = ["no h2 network","with h2 network" ]

plt.figure(figsize=(10, 6))
df.plot()
plt.title("Electrolyzer system load duration curve")
plt.ylabel("Electrolyzer load [MWh]")
plt.xlabel("Fraction of total time")
plt.legend()
plt.show()

### Storage Behavior

#### Feed-in and feed-out pattern

In [ ]:
# Energy as calculated by the OPF.
n.stores_t.p[index_h2stores].sum(axis=1).plot()
plt.show()

#### Storage level over time

In [ ]:
# Energy as calculated by the OPF.
n.stores_t.e[index_h2stores].sum(axis=1).plot()
plt.show()

In [ ]:
# storage level of 10 largest stores
index = n.stores.e_nom_opt[index_h2stores].sort_values(ascending=False).head(10).index

# yearly plot
for store in index:
    fig, ax = plt.subplots()
    n.stores_t.e[store].plot(ax=ax, ylabel=store)

## Ideas:
- Calc capacity factors / system load duration curves for electrolysis in different regions and try to investigate in differences (amount of wind generation, solar) Is only excess electrolysis consumed by electrolysis?
- Try to determine connection between electrolysis and excess solar / wind production
- Try to determine percent / amount of excess electricity that is used to produce hydrogen. (Basically all energy from renewables that went into electrolysis?)
- pie chart of percentage of electricity that is used directly, used for Electrolysis, battery, ... (startup script)
- Reproduce graph from Victoria, Zhu et al. 2019 – The role of storage technologies (in hydrogen word document); try to find patterns within the charging of battery and the charging with hydrogen (electrolysis -> fuel cell)
-